In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

GPT4='gpt-4'
GPT35='gpt-3.5-turbo'
API_KEY=os.environ.get('API_KEY') 
openai.api_key=API_KEY



In [ ]:
def testSamplePrompt():
    response=openai.ChatCompletion.create(
        model=GPT35,
        messages=[{
            "role":"user",
            "content":"Generate a list of numbers from 1 to 50"
        }],
        stream=True
    )
    return response
resp=testSamplePrompt()
for res in resp:
    try:
        content=res["choices"][0]["delta"]["content"]
        print(content,end='')
    except:
        break
 

In [2]:
import time
import os
from retrievalFunction import retrievalFunction
from responseGenerator import (
    generateInitialObservations,
    generateObservations,
    generateConversation,
)
from textBasedSimulation import startConversation,getBaseDescription,updateBaseDescription


from concurrent.futures import ThreadPoolExecutor
from collections import deque
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
load_dotenv()

# Constants
DATABASE_NAME = "LLMDatabase"
DATABASE_URL = os.environ.get("DATABASE_URL")
COLLECTION_USERS = "Users"
COLLECTION_MEMORY_OBJECTS = "MemoryObjects"

# Basic objects for the Database.
client = MongoClient(DATABASE_URL)
LLMdatabase = client[DATABASE_NAME]
userCollection = LLMdatabase[COLLECTION_USERS]
memoryObjectCollection = LLMdatabase[COLLECTION_MEMORY_OBJECTS]


c:\grad\research apprenticeship\abdulaziz_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if __name__ == "__main__":
    pastObservations = deque()
    # Get username.
    userName = input("Please enter the username of character: ")

    # Check for existing user.
    existingUser = userCollection.find_one({"Username": userName})

    if existingUser:
        print(f"Welcome back! {userName} \nContinue where you left off")
    else:
        # Collect the description details.
        description = getBaseDescription()

        # Insert the userData to the Users collection.
        userData = {"Username": userName, "Description": description}
        userCollection.insert_one(userData)

        # Time the function call and fetch the results.
        startTime = time.time()
        observationList = generateInitialObservations(userName, description).split("\n")
        endTime = time.time()
        print(
            f"Time taken for the observation generation by GPT : {endTime-startTime:.2f} "
        )

        # Generate the memory object data and push it to the memory objects collection.
        updateBaseDescription(userName, observationList)
        print("User created successfully!")

    startConversation(userName)
    client.close()


Welcome back! John Lin 
Continue where you left off
John Lin :
Eddy is my son. He studies music theory. Is there anything specific you would like to know about him?
Time taken for the conversation generation by GPT : 1.81
